기존 모델에서 이진분류방식을 적용해서 분할
-> 데이터 불균형 해소방안

# 데이터 가공

- 범주형 데이터 스코어링 후 수치화 / 나머지 비중요 범주형 데이터는 라벨인코딩
- 데이터특성, 도메인 지식 (EDA, RFM 분석 결과 바탕) 파생변수 생성
- 전체 상관계수 & VIF 확인 후 나머지 데이터 가공 (가공방법 : 상관계수 기반 파생변수 생성 - 변수 특성 (B : 해당월, R : 해당기간까지의 누적) 반영)
- 전체 VIF 확인후 수치 너무 높은 컬럼은 제거 (대표값 하나씩만 남김)

In [5]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import platform

# 시스템이 Windows일 경우
if platform.system() == 'Windows':
    plt.rc('font', family='Pretendard')
# macOS일 경우 예시:
elif platform.system() == 'Darwin':
    plt.rc('font', family='AppleGothic')

# 마이너스 부호 깨짐 방지
plt.rcParams['axes.unicode_minus'] = False

In [6]:
from modules.data_loader import load_and_process
from modules.feature_selector import stage_feature_map

file_path = "../../data/통합_train_데이터.parquet"
test_path = "../../data/통합_test_데이터.parquet"

stage_name = "e" # <- 변경해서 사용
save_path = "./models"

### 데이터 로드

In [8]:
from modules.feature_selector import pca_cols, ab_feat, cd_baseline, selected_cols
from modules.feature_selector import stage_feature_map

e_cols = selected_cols
train_e, e_cols = load_and_process(file_path, stage_name)

데이터 불러오기 완료.
[연회비발생카드수_B0M] 인코딩 완료
[한도증액횟수_R12M] 인코딩 완료
[이용금액대] 중간값 인코딩 완료
[할인건수_R3M] 인코딩 완료
[할인건수_B0M] 인코딩 완료
[방문횟수_PC_R6M] 인코딩 완료
[방문횟수_앱_R6M] 인코딩 완료
[방문일수_PC_R6M] 인코딩 완료
✅ 범주형 인코딩 완료


### 상관계수, VIF 점검
| score를 높이기위해 상관계수 기반 , 인사이트 기반으로 추가 파생변수 생성
| VIF 기준 너무 높은 변수만 제거 (but 중요도 높은 일부 컬럼 제거대상 제외)

In [10]:
from modules.preprocess_utils import get_high_correlation_pairs
# 인코딩 및 수치형 컬럼만 포함된 DataFrame
df_numeric = train_e.copy()

# 상관계수 0.8 이상 쌍만 표로 추출
high_corr_table = get_high_correlation_pairs(df_numeric, threshold=0.96)

# 결과 출력
import pandas as pd
# pd.set_option('display.max_rows', 100)
# display(high_corr_table)

In [9]:
from modules.feature_selector import generate_e_features
train_e = generate_e_features(train_e)

In [ ]:
# from modules.preprocess_utils import fast_vif

# # 수치형 변수만
# X = train_e.select_dtypes(include=['int64', 'float64']).drop(columns=['ID', 'Segment'], errors='ignore')

# vif_df = fast_vif(X, verbose=False)  # 로그 출력 끄고 실행
# top_vif_20 = vif_df.sort_values(by='VIF', ascending=False).head(20)
# print(top_vif_20)

### VIF & 상관계수 기준으로 파생변수 생성

In [11]:
from modules.preprocess_utils import auto_reduce_vif_features_fast
from modules.preprocess_utils import remove_inf_div_features_fast
from modules.preprocess_utils import auto_clean_high_vif_features

# 1. 상관계수 기반 VIF 파생 생성
df_vif, derived_cols, top_vif = auto_reduce_vif_features_fast(train_e, high_corr_table, top_k=30)

# 2. 생성된 _div_ 파생변수 중 inf/NaN 제거
df_vif_clean, removed_div_cols = remove_inf_div_features_fast(df_vif)

# 3. 주요 VIF 유발 변수 정제 + 파생 변수로 요약
df_final, removed_highvif_cols, created_highvif_cols = auto_clean_high_vif_features(df_vif_clean)

# ✅ 최종 feature 후보군
all_derived = derived_cols + created_highvif_cols
print(f"\n🎯 최종 사용 가능한 파생변수 수: {len(all_derived)}")


📊 [VIF 상위 30개]
           feature          VIF
        신용이용건수_증가량          inf
       이용건수_신용_R6M          inf
      이용건수_신용_R12M          inf
      이용건수_신판_R12M 67306.333520
     이용건수_일시불_R12M 18097.676276
  잔액_신판최대한도소진율_r3m   319.745854
잔액_신판ca최대한도소진율_r3m   254.675915
  잔액_신판평균한도소진율_r3m   238.253804
  잔액_신판평균한도소진율_r6m   230.703912
잔액_신판ca최대한도소진율_r6m   207.702156
잔액_신판ca평균한도소진율_r6m   206.793101
  잔액_신판최대한도소진율_r6m   195.494910
잔액_신판ca평균한도소진율_r3m   160.740325
        잔액_카드론_B1M   122.795660
   포인트_마일리지_건별_R3M   104.356991
        잔액_카드론_B0M    99.664666
      이용금액_일시불_R3M    83.094039
   포인트_마일리지_건별_B0M    79.105772
        방문횟수_앱_B0M    70.278901
       이용건수_신용_B0M    64.620425
        방문일수_앱_B0M    63.405761
      잔액_현금서비스_B1M    61.343282
      이용금액_일시불_R6M    59.233115
      이용금액_일시불_B0M    57.230557
      마일_적립포인트_R3M    49.099322
      카드이용한도금액_B1M    48.391739
      잔액_현금서비스_B0M    44.092891
       방문횟수_PC_B0M    41.756527
       방문일수_PC_B0M    41.755432
          평잔_할부_3M    41

In [ ]:
from modules.preprocess_utils import fast_vif_cleaner

# 2. 전체 컬럼 중 ID, Segment 제외한 파생변수만 추출
all_derived_features = [col for col in derive_train.columns if col not in ["ID", "Segment"]]

# 3. 컬럼 수 제한 (RAM 보호용) → 상위 50개만 사용
important_cols = all_derived_features[:50]

# 4. 선택한 변수만 추출
X = derive_train[important_cols].copy()

# 5. fast VIF 계산 및 변수 제거
X_cleaned, removed_cols, final_vif = fast_vif_cleaner(X, vif_threshold=100)

# 6. 결과 확인
print("✅ 제거된 VIF 변수 수:", len(removed_cols))
print("📊 최종 VIF 상위 5개:")
print(final_vif.head())

In [ ]:
import pandas as pd
# 1. 제거할 컬럼 리스트
remove_cols = ["방문횟수_PC_B0M", "카드이용한도금액_B1M", "이용카드수_체크_가족"]

# 2. 최종 컬럼 구성 (ID, Segment 포함)
selected_features = ["ID", "Segment"] + [
    col for col in derive_train.columns
    if col not in remove_cols + ["ID", "Segment"]
]
# 리스트를 DataFrame으로 변환
feature_df = pd.DataFrame({"selected_features": selected_features})

# CSV로 저장
feature_df.to_csv("최종_사용_피처목록.csv", index=False, encoding="utf-8-sig")

# 3. 최종 데이터프레임 생성
final_df = derive_train[selected_features].copy()

# 4. Parquet 파일로 저장
final_df.to_parquet("최종_사용_데이터프레임.parquet", index=False)